### Authenticate and get headers

In [ ]:
import requests
import pandas as pd
from io import StringIO
from _includes import config
from _includes import helper

config_dict = config.config_dict

api = helper.APIHelper(config_dict)
base_site_url = f'{api.server}/api/{api.api_version}/sites/{api.site_id}'
base_api_url = f'{api.server}/api/-'
headers = api.headers

### List all definitions and metrics

In [ ]:
metric_df = api.get_pulse_metric_definitions_and_metrics_df(filter='DataDev')
metric_df

### Make a lookup dictionary

In [ ]:
metric_dict = {
    (row['definition_name'], row['granularity'], row['range']): row['metric_id']
    for _, row in metric_df.iterrows()
}

### Sample CSV Data
### For the CSV Files / Data Source you plan to be a source file for automated subscription, I highly recommend you to use simple words (YEAR/QUARTER/MONTH) or (CURRENT/LAST) ... and then you transform the simple word to correct value later.

In [ ]:
df_csv = pd.read_csv('Step-9.csv')
df_csv

In [ ]:
df = df_csv.copy()
df['granularity'] = 'GRANULARITY_BY_' + df['granularity'].str.upper()
df['range'] = df['range'].str.upper().replace({
    'LAST': 'RANGE_LAST_COMPLETE',
    'CURRENT': 'RANGE_CURRENT_PARTIAL'
})
df

### Translate Email / Username to User ID

In [ ]:
user_dict = api.get_user_dict()

### The result we will use for subscription

In [ ]:
map_df = df.copy()
map_df['metric_id'] = map_df.apply(lambda row: metric_dict.get((row['definition_name'], row['granularity'], row['range'])), axis=1)
map_df['user_id'] = map_df['user'].apply(lambda user: user_dict.get(user))
map_df

In [ ]:
for i,row in map_df.iterrows():

    definition_name = row['definition_name']
    granularity = row['granularity']
    range = row['range']
    user = row['user']

    metric_id = row['metric_id']
    user_id = row['user_id']

    payload = {
        "metric_id": metric_id,
        "follower": {
            "user_id": user_id
        }
    }

    try:
        print(f'Subscribe {definition_name} / {granularity} / {range} : For {user} ')
        endpoint = f'{base_api_url}/pulse/subscriptions'
        api_response = requests.post(endpoint, headers=headers, json=payload)
        api_response.raise_for_status()
        api_response_json = api_response.json()
        metric_id = api_response_json.get('subscription').get('id')
        print('Subscription created successfully.')
        print('Subscription ID:')
        print(metric_id)
    except requests.exceptions.RequestException as e:
        print(f'Request failed: {e}')
        if api_response.content:
            print('Response content:', api_response.content.decode())